In [ ]:
# Import basic libraries to handle data and build the model
import zipfile
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [ ]:
#!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

In [ ]:
# Set device (GPU if available)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
os.chdir("/content/Qualcomm-DL-Hackathon/train")
extract_dir = "/content/Qualcomm-DL-Hackathon/train/"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("images part-1.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
with zipfile.ZipFile("images part-2.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Contents extracted to {extract_dir}")

In [ ]:
os.rename('/content/Qualcomm-DL-Hackathon/train/images part-1', '/content/Qualcomm-DL-Hackathon/train/images')

In [ ]:
import shutil
for x in os.listdir('/content/Qualcomm-DL-Hackathon/train/images part-2'):
  source_path = os.path.join('/content/Qualcomm-DL-Hackathon/train/images part-2', x)
  shutil.move(source_path, '/content/Qualcomm-DL-Hackathon/train/images/')

In [ ]:
#shutil.rmtree('/content/Qualcomm-DL-Hackathon/train/images')
shutil.rmtree('/content/Qualcomm-DL-Hackathon/train/images part-2')

In [ ]:
from PIL import Image
os.getcwd()

In [ ]:
#reading images to check size
count = 0
for x in os.listdir('/content/Qualcomm-DL-Hackathon/train/images/'):
  img = Image.open('/content/Qualcomm-DL-Hackathon/train/images/'+x)
  count+=1
count

In [ ]:
img.size

In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.shape

In [ ]:
from sklearn.preprocessing import LabelBinarizer

train_image_names = train_data['image_names'].values
test_image_names = test_data['image_names'].values

# Initialize the LabelBinarizer
train_labels = train_data['emergency_or_not'].values

# Encode labels
#label_binarizer = LabelBinarizer()
#train_labels = label_binarizer.fit_transform(train_labels)
train_labels


In [ ]:
import numpy as np

train_image_paths = np.array([os.path.join('/content/Qualcomm-DL-Hackathon/train/images', file_name) for file_name in train_image_names])
test_image_paths = np.array([os.path.join('/content/Qualcomm-DL-Hackathon/train/images', file_name) for file_name in test_image_names])


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])
test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [ ]:
train_dataset = CustomDataset(image_paths=train_image_paths, labels=train_labels, transform=train_transforms)
test_dataset = CustomDataset(image_paths=test_image_paths, transform=test_transforms)

In [ ]:
# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

In [ ]:
train_size, val_size

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define the CNN model architecture (no modular code)
conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
fc1 = nn.Linear(128 * 16 * 16, 256)
fc2 = nn.Linear(256, 2)  # 2 classes (emergency non-emergency)

In [ ]:
# Initialize the model
model = nn.Sequential(
    conv1,
    nn.ReLU(),
    pool,
    conv2,
    nn.ReLU(),
    pool,
    conv3,
    nn.ReLU(),
    pool,
    nn.Flatten(),
    fc1,
    nn.ReLU(),
    nn.Dropout(0.3),
    fc2,
    nn.Softmax()
)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training the model
num_epochs = 7
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    total = 0
    correct = 0


    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track the loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

In [ ]:
# Hyperparameters to tune
learning_rates = [0.001, 0.01]
batch_sizes = [16, 32]
dropout_rates = [0.3, 0.5]
hidden_layers = [1, 2,3]  # 1 hidden layer or 2 hidden layers
hidden_units = [128, 256]  # Size of each hidden layer
num_epochs = 10

# Placeholder for the best model and hyperparameters
best_model = None
best_accuracy = 0
best_params = {}

# Grid search loop over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
        for dropout_rate in dropout_rates:
            for num_hidden_layers in hidden_layers:
                for hidden_units_per_layer in hidden_units:

                    # Recreate data loaders with the current batch size
                    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
                    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                    # Define the CNN model architecture with the current number of hidden layers and dropout rate
                    conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
                    conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
                    conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
                    pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                    fc_layers = []

                    # Create hidden layers based on the number of hidden layers
                    input_size = 128 * 16 * 16  # Image size after convolution and pooling
                    for i in range(num_hidden_layers):
                        fc_layers.append(nn.Linear(input_size, hidden_units_per_layer))
                        fc_layers.append(nn.ReLU())
                        fc_layers.append(nn.Dropout(dropout_rate))
                        input_size = hidden_units_per_layer  # Update input size for the next layer

                    fc_layers.append(nn.Linear(input_size, 5))  # Output layer for 5 classes (cycling, dancing, drinking, eating, sitting)

                    # Combine layers into a model
                    model = nn.Sequential(
                        conv1,
                        nn.ReLU(),
                        pool,
                        conv2,
                        nn.ReLU(),
                        pool,
                        conv3,
                        nn.ReLU(),
                        pool,
                        nn.Flatten(),
                        *fc_layers
                    ).to(device)

                    # Define the loss function and optimizer for this combination of hyperparameters
                    criterion = nn.CrossEntropyLoss()
                    optimizer = optim.Adam(model.parameters(), lr=lr)

                    # Train the model
                    for epoch in range(num_epochs):
                        model.train()
                        running_loss = 0.0
                        correct = 0
                        total = 0

                        for inputs, labels in train_loader:
                            inputs, labels = inputs.to(device), labels.to(device)

                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()

                            running_loss += loss.item()
                            _, predicted = torch.max(outputs, 1)
                            total += labels.size(0)
                            correct += (predicted == labels).sum().item()

                        epoch_loss = running_loss / len(train_loader)
                        epoch_accuracy = 100 * correct / total
                        print(f"Epoch [{epoch+1}/{num_epochs}] for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: Loss={epoch_loss:.4f}, Accuracy={epoch_accuracy:.2f}%")

                    # Evaluate on the validation set after training
                    model.eval()
                    val_correct = 0
                    val_total = 0
                    with torch.no_grad():
                        for inputs, labels in val_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            outputs = model(inputs)
                            _, predicted = torch.max(outputs, 1)
                            val_total += labels.size(0)
                            val_correct += (predicted == labels).sum().item()

                    val_accuracy = 100 * val_correct / val_total
                    print(f"Validation Accuracy for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: {val_accuracy:.2f}%")


In [ ]:
# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_correct = 0
val_total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        #inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_accuracy = 100 * val_correct / val_total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

In [ ]:
# Generate predictions for test images
model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Print or save the predictions
print(predictions)